In [1]:
from utils import build_url, load_json, save_json, call_with_retry, save_execution_state, load_execution_state
import os, shutil

API_KEY_FILE = 'api_key.txt'
REGION = 'euw1'
BASE_URL = 'api.riotgames.com'

QUEUE_TYPE = [420, 430, 440]  # Ver https://developer.riotgames.com/docs/lol#general_queue-ids

with open(API_KEY_FILE) as f:
    api_key = f.read()

execution_cache_dir = 'current_execution'
games_ids_state = f'{execution_cache_dir}/games.pkl'
accounts_state = f'{execution_cache_dir}/accounts.pkl'

os.makedirs(execution_cache_dir, exist_ok=True)
MIN_DELAY = 20/100 # esperar 20ms entre peticiones
delay = MIN_DELAY

headers = {
  'X-Riot-Token': api_key
}

accounts = load_json('outputs/accounts.json')
print(accounts[0:5])
accounts_ids = list(map(lambda a: a['account'], accounts))
print(accounts_ids[0:10])

[{'summoner': 'DEXdsT_kkSFvatrJWf6BvzxoJoKWTrEJxY494i_AfBLs9Ho', 'account': 'cpDa6rnHFIQP33_P-Thda6DBEFZQQCFYNIbcnClYbXKGrEc'}, {'summoner': 'Fzd0uBGACARSDJKs4QiIDZHo1XUZjYNGm7j_1Czzd1Qjtqw', 'account': 'ypbT3klYhns-5YIPHHeZMm_YSLq06hGEqC1DU9qB_oFPkzI'}, {'summoner': 'XUahKkY6p3V8kh7cOP4Y2puoZBrql7NCk5KLCqQllvpXqUY', 'account': 'ciMNptPkXwuQcRPekdtZgf8BSZ4hRPuN_lmeNPGfSr9vSLg'}, {'summoner': '8QE8GfqXBMw9-FwRZ2Q5sbS-5M3KxrmWPPNrG6arRDPT1Cs', 'account': 'XpcM677yGPo2MQvoDUzBxjmE6JwzUlU-qtHDfikrWBmxL5Q'}, {'summoner': 'CDrf2ct4LcmOmfb8jz-nb-P_W2NTiO0lkaJZALvJAetk0IiRwzKB2LKJHA', 'account': 'C7Ul2PgEW9by___3zSy7VFe-lN10Q8r-fiTe2kx4muvD-DTc-MHVn7kY'}]
['cpDa6rnHFIQP33_P-Thda6DBEFZQQCFYNIbcnClYbXKGrEc', 'ypbT3klYhns-5YIPHHeZMm_YSLq06hGEqC1DU9qB_oFPkzI', 'ciMNptPkXwuQcRPekdtZgf8BSZ4hRPuN_lmeNPGfSr9vSLg', 'XpcM677yGPo2MQvoDUzBxjmE6JwzUlU-qtHDfikrWBmxL5Q', 'C7Ul2PgEW9by___3zSy7VFe-lN10Q8r-fiTe2kx4muvD-DTc-MHVn7kY', 'mw0YEQVwXJDuQ_F7WiyLqssFON_S5H0k7lfSXxbzRJg_4ARg1MhfmWKn', 'a_DtSjw7ZpQgGknDlT

In [2]:
endpoint = '/lol/match/v4/matchlists/by-account/{}'
params = {
    'queue': QUEUE_TYPE,
}

games_ids = load_execution_state(games_ids_state, set())
accounts_with_matches = load_execution_state(accounts_state, set())
accounts_ids = set(accounts_ids) - accounts_with_matches

for account_id in accounts_ids:
    full_url = build_url(REGION, BASE_URL, endpoint.format(account_id))
    response = call_with_retry(full_url, headers, params)
    for match in response['matches']:
        games_ids.add(match['gameId'])
    save_execution_state(games_ids, games_ids_state)
    accounts_with_matches.add(account_id)
    save_execution_state(accounts_with_matches, accounts_state)
        
save_json(list(games_ids), 'games_ids.json')

In [3]:
# Limpiar estados
shutil.rmtree(execution_cache_dir)